In [1]:
%%configure -f
{
 "executorCores": 4, 
 "executorMemory": "47696M", 
 "conf": {"spark.default.parallelism": 1000,
          "spark.sql.shuffle.partitions": 1000,
          "spark.task.cpus": 1
         }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1575985650678_0001,pyspark,killed,Link,,


In [2]:
from gensim.utils import lemmatize, tokenize
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from langdetect import detect
import string
from itertools import islice
import numpy as np
import smart_open
from collections import Counter, defaultdict
import math
import spacy

from pyspark.ml.feature import (HashingTF,
                                IDF,
                                Tokenizer,
                                StopWordsRemover,
                                CountVectorizer,
                                StringIndexer,
                                OneHotEncoderEstimator,
                                VectorAssembler,
                                VectorSizeHint,
                                StandardScaler,
                                PCA
                               )
from pyspark.ml import Pipeline, Transformer

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

from pyspark.ml.linalg import Vectors, VectorUDT

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1575985650678_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
word2id_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/common/bow_description_not_lemmatized"
data_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/data_desc_only/raw_company_features_bow_description_not_lemmatized"
model_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/model/probabilistic_not_lemmatized/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
companies_df = (spark.read.load(f"{data_path}/raw_company_features_bow_description_not_lemmatized")
                .join(spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/company_data_denormalized"),
                      on=["entity_id"]
                     )
                .select("entity_id", "country_of_incorporation", F.explode("bow_description_not_lemmatized"))
                .select("entity_id", "country_of_incorporation", F.col("key").alias("word"))
               )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
num_companies = companies_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
companies_in_country = {row.country_of_incorporation: row.n_companies
                        for row in 
                        companies_df
                        .groupBy("country_of_incorporation")
                        .agg(F.countDistinct("entity_id").alias("n_companies"))
                        .collect()
                       }

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
companies_word = {row.word: row.n_companies
                          for row in 
                          companies_df.groupBy("word")
                                      .agg(F.countDistinct("entity_id").alias("n_companies"))
                                      .filter(F.col("n_companies") > 15)
                                      .collect()
                         }

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
companies_word_country = {(row.country_of_incorporation, row.word) : row.n_companies
                          for row in 
                          companies_df.groupBy("country_of_incorporation", "word")
                                      .agg(F.countDistinct("entity_id").alias("n_companies"))
                                      .collect()
                         }

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
specific_words = defaultdict(list)
for (country,word), n_companies_word_country in companies_word_country.items():
    if word not in companies_word:
        continue
    background_word = companies_word[word]/num_companies
    country_word = n_companies_word_country/companies_in_country[country]
    specific_words[country].append((word, country_word/background_word))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
for country in specific_words:
    specific_words[country] = list(sorted(specific_words[country], key=lambda el: -el[1]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
for country,words in islice(specific_words.items(),10):
    print(f"{country},{','.join([el[0] for el in islice(words, 10)])}")
    print("\n --- \n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Ireland,laoghaire,ilfc,sligo,navan,dublin,swords,galway,limerick,celestial,shannon

 --- 

United States,thomasville,kingsport,bastrop,amortizations,henrico,bldng,swer,displace,bozeman,everglades

 --- 

Australia,bendigo,leederville,nedlands,subiaco,yilgarn,cairns,wiluna,dandenong,leonora,mulgrave

 --- 

China,suqian,shangrao,technologyco,guodian,huasheng,jinzhong,chizhou,longyan,quzhou,heyuan

 --- 

Germany,nach,hafen,tuttlingen,leinfelden,bensheim,heidenheim,unternehmen,zusammenhaengenden,grundstuecke,einer

 --- 

Luxembourg,bertrange,leudelange,strassen,duchy,à,luxembourg,foyer,á,frght,electrometallurgical

 --- 

United Kingdom,caledonian,gerrards,flintshire,coldfield,swinton,mfrs,ballymena,morecambe,islington,scunthorpe

 --- 

Norway,vestfold,bodø,boligkreditt,lillehammer,ulsteinvik,ålesund,hamar,akershus,kongsvinger,molde

 --- 

Pakistan,modaraba,faisalabad,karachi,rawalpindi,lahore,islamabad,habib,ijarah,pakistan,kohinoor

 --- 

Bermuda,renaissancere,partnerre,validus,ber

In [68]:
def print_topic(topic):
    for el in islice(sorted(enumerate(phi[topic]), key=lambda a: -a[1]), 100):
        print(id2word[el[0]], el[1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
word2id = {}
id2word = {}
with smart_open.open(f"{word2id_path}/word2id.csv", "r") as f:
    for line in f:
        word,idd = line.strip().split(",")
        word2id[word] = int(idd)
        id2word[int(idd)] = word

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
industry2idx = {}
idx2industry = {}
with smart_open.open(f"{model_path}/simple_LDA/industries", "r") as f:
    for line in f:
        toks = line.strip().split(",")
        industry,idd = ",".join(toks[:-1]), toks[-1]
        industry2idx[industry] = int(idd)
        idx2industry[int(idd)] = industry

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
with smart_open.open(f"{model_path}/simple_LDA/parameters/phi.np", "rb") as f:
    phi = np.load(f, allow_pickle=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
bg_topic = phi[-1,:]
top_bg_words = []
for i,val in enumerate(bg_topic):
    top_bg_words.append((id2word[i], val))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
print(idx2industry[802])
print_topic(802)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Motion picture production and allied services
shaanxi 0.06957796757194275
cultural 0.06957796757151215
investment 0.0695349534623082
june 0.06946417185146923
holding 0.06942290831886092
china 0.06920831642978925
group 0.06837598288413903
industry 0.06820895760703823
co 0.06786573182384527
on 0.06770592802347888
ltd 0.06735091423258478
founded 0.0647481507528813
based 0.0625159502672862
was 0.05965097554951865
is 0.05009864806479187
in 0.006692474597704705
and 9.212707038548928e-10
a 0.0
aa 0.0
aaa 0.0
aac 0.0
aachen 0.0
aag 0.0
aai 0.0
aak 0.0
aalberts 0.0
aalborg 0.0
aalen 0.0
aalsmeer 0.0
aalst 0.0
aalter 0.0
aam 0.0
aan 0.0
aankoop 0.0
aannemer 0.0
aanverwante 0.0
aap 0.0
aar 0.0
aarau 0.0
aareal 0.0
aareon 0.0
aarhus 0.0
aaron 0.0
aarp 0.0
aarsleff 0.0
aartselaar 0.0
aastra 0.0
aat 0.0
aav 0.0
ab 0.0
aba 0.0
ababa 0.0
abacus 0.0
abadi 0.0
abalone 0.0
abandon 0.0
abandoned 0.0
abandonment 0.0
abap 0.0
abarth 0.0
abashiri 0.0
abatement 0.0
abattoir 0.0
abattoirs 0.0
abb 0.0
abba 0.0


In [57]:
tst = np.array([[1,2],
                [4,201],
                [100, 200],
               ])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
word_max = phi.argmax(axis=0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
word_max[word2id['ltd']]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

802

In [65]:
word_max.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(53709,)